In [ ]:
#include("src/helper.jl")
#using .helper
using Eirene
using Ripserer
using CSV
using TiffImages
using Images
using NPZ
using Plots
using PersistenceDiagrams
using Measures
using Distributions
using MultivariateStats
using LinearAlgebra
using Random
using StatsBase
using JLD2
using FileIO
using PersistenceDiagrams
using DelimitedFiles
using UMAP


In [ ]:
include("../src/ECM_TDA.jl")
using .ECM_TDA

In [ ]:
# color palettes
c1 = "#E64B35FF" # red
c2 = "#4DBBD5FF" # blue
c3 = "#00A087FF" # green
c4 = "#3C5488FF" # dark blue
c5 = "#F39B7FFF" # salmon
c6 = "#8491B4FF"

In [ ]:
# assign colors to cell types
c_cancer = "#2EC4B6" # light blue
c_leukocytes = "#CDCDCD"  # grey
c_fibroblast= "#7209B7" # purple
c_ECM = "#EF476F" # pink
#FFC300 : yellow

celltype_colors = Dict("cancer" => c_cancer, 
                        "leukocytes"=> c_leukocytes, 
                        "fibroblast"=> c_fibroblast);

# 1. Plot example (check alignment)

### original ROI

In [ ]:
LTX = "001"
Da = "232"
idx = 1

# load ECM image
image_path = "data/4000x4000/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
img = Array(Images.load(image_path));

# load sampled ECM
ecm_points_path = "data/4000x4000_combined/ECM_sampled/points_CSV/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv" 
ECM_points = CSV.read(ecm_points_path)

# load cells
cell_path = "data/4000x4000_combined/subregion_cells/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv"
cells = CSV.read(cell_path);

In [ ]:
gr()
p1 = plot(Gray.(img), frame = :box, ticks = [], size = (300, 300), background_color=:transparent, foreground_color=:black)
p2 = scatter(ECM_points[:,1], ECM_points[:,2], yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], markersize = 2, background_color=:transparent, foreground_color=:black)
plot(p1, p2, size = (600, 300))

In [ ]:
cell_types =["cancer", "leukocytes"]
plot_array = Any[]
for ct in cell_types
    # get locations of cell type
    cell_ct = cells[cells.class .== ct, :]
    c = celltype_colors[ct]
    p_ct = scatter(cell_ct.x, cell_ct.y,
                 markersize = 2,
                 yflip = true,
                 label = ct,
                 markerstrokewidth = 0.2,
                 frame = :box,
                 ticks = [],
                 c = c) 
    push!(plot_array, p_ct)
end

plot(plot_array..., layout = grid(1,2), size = (800, 400))

# 2. Compute PH for cancer cells and leukocytes

In [ ]:
dir = "data/4000x4000_combined/subregion_cells/"
csv_files = [item for item in walkdir(dir)][1][3:end][1];

In [ ]:
PD0_cancer = Dict()
PD0_leukocytes = Dict()
PD1_cancer = Dict()
PD1_leukocytes = Dict()

for file in csv_files

    cells = CSV.read(dir * file)
    cancer_PD0, cancer_PD1 = run_PH_cell_type(cells; celltype = "cancer")
    leukocytes_PD0, leukocytes_PD1 = run_PH_cell_type(cells; celltype = "leukocytes")
    
    # save
    filename = split(file, ".")[1]
    writedlm("data/4000x4000_combined/cells_PD/cancer/PD0/" * filename * ".csv", cancer_PD0, ",")
    writedlm("data/4000x4000_combined/cells_PD/cancer/PD1/" * filename * ".csv", cancer_PD1, ",")
    writedlm("data/4000x4000_combined/cells_PD/leukocytes/PD0/" * filename * ".csv", leukocytes_PD0, ",")
    writedlm("data/4000x4000_combined/cells_PD/leukocytes/PD1/" * filename * ".csv", leukocytes_PD1, ",")

    
    # combine to dictionary
    PD0_cancer[filename] = cancer_PD0
    PD1_cancer[filename] = cancer_PD1
    PD0_leukocytes[filename] = leukocytes_PD0
    PD1_leukocytes[filename] = leukocytes_PD1
end

In [ ]:
save("data/4000x4000_combined/cells_PD/PD.jld2",
    "PD0_cancer", PD0_cancer,
    "PD1_cancer", PD1_cancer,
    "PD0_leukocytes", PD0_leukocytes,
    "PD1_leukocytes", PD1_leukocytes)

Compute persistence image

In [ ]:
PH0_cancer = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD0_cancer)
PH1_cancer = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD1_cancer)
PH0_leukocytes = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD0_leukocytes)
PH1_leukocytes = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD1_leukocytes);


In [ ]:
PI0_cancer= ECM_TDA.compute_PI(PH0_cancer)
PI1_cancer = ECM_TDA.compute_PI(PH1_cancer)
PI0_leukocytes = ECM_TDA.compute_PI(PH0_leukocytes)
PI1_leukocytes = ECM_TDA.compute_PI(PH1_leukocytes)


In [ ]:
save("data/4000x4000_combined/cells_PD/PI.jld2",
    "PI0_cancer", PI0_cancer,
    "PI1_cancer", PI1_cancer,
    "PI0_leukocytes", PI0_leukocytes,
    "PI1_leukocytes", PI1_leukocytes)

In [ ]:
cancer0_max = get_PD0_max(PD0_cancer)
leukocytes0_max = get_PD0_max(PD0_leukocytes)

cancer1_max = get_PD1_max(PD1_cancer)
leukocytes1_max = get_PD1_max(PD1_leukocytes)

In [ ]:
# save the max values

save("data/4000x4000_combined/cells_PD/PD_max.jld2", 
    "cancer0_max", cancer0_max,
    "cancer1_max", cancer1_max,
    "leukocytes0_max", leukocytes0_max,
    "leukocytes1_max", leukocytes1_max)
